# Aug ??, 2022 (cos sim 8 x conds: 512)

**Motivation**:  <br>

In [ ]:
# HIDE CODE


import os
import sys
from pprint import pprint
from copy import deepcopy as dc
from os.path import join as pjoin
from scipy.ndimage import gaussian_filter
from IPython.display import display, IFrame, HTML

# tmp & extras dir
git_dir = pjoin(os.environ['HOME'], 'Dropbox/git')
extras_dir = pjoin(git_dir, 'jb-Ca-fMRI/_extras')
fig_base_dir = pjoin(git_dir, 'jb-Ca-fMRI/figs')
tmp_dir = pjoin(git_dir, 'jb-Ca-fMRI/tmp')

# GitHub
sys.path.insert(0, pjoin(git_dir, '_Ca-fMRI'))
from figures.fighelper import *
from analysis.final import *
from utils.render import *

# warnings, tqdm, & style
warnings.filterwarnings('ignore', category=DeprecationWarning)
from tqdm.notebook import tqdm
%matplotlib inline
set_style()

## Fig dir

In [ ]:
fig_dir = pjoin(fig_base_dir, 'paper2.0', '8x')
os.makedirs(fig_dir, exist_ok=True)
print(fig_dir)

merge_thres = {
    128: 0.376,
    512: 0.444,
}
kws_fig = {
    'transparent': True,
    'bbox_inches': 'tight',
    'dpi': 300,
}

## Prepare (512)

In [ ]:
nn = 512
mice = Mice(nn)
*_, props_ca = default_props(mice)
bt = BootStrap(**props_ca)

## Sim results (just show)

In [ ]:
%%time
looper = itertools.product([3, 7, 20], [10, 15, 20, 25])

for num_k, perc in looper:
    try:
        results = similiarity_results(
            mice=mice,
            perc=perc,
            num_k=num_k,
            n_resamples=None,
            merge_thres=merge_thres[nn]
            if num_k == 7 else None,
        )
        print('-' * 25 + f"    K = {num_k},  d = {perc}%    " + '-' * 25)
        _ = bt.show(results['pi_sorted'], **show_kws(num_k))
        print('-' * 73)
        print('\n\n')
    except AttributeError:
        continue

In [ ]:
num_k, perc

## Sim results (full)

In [ ]:
from figures.fig_ocs import _scores_arr

In [ ]:
%%time

scores, std_err = {}, {}
pvalue = np.zeros((3, 4))
statistic = np.zeros((3, 4))
for i, num_k in enumerate([3, 7, 20]):
    res = {}
    for j, perc in enumerate([10, 15, 20, 25]):
        results = similiarity_results(
            mice=mice,
            perc=perc,
            num_k=num_k,
            n_resamples=int(1e2),
            merge_thres=merge_thres[nn]
            if num_k == 7 else None,
        )
        res[perc] = results
        test = list(results['tests_overall'].values())[0]
        pvalue[i, j] = test.pvalue
        statistic[i, j] = test.statistic
    s, e = _scores_arr(res, True)
    scores[num_k] = s
    std_err[num_k] = e

In [ ]:
{k: v.shape for k, v in scores.items()}

shape: anim x perc x cond

In [ ]:
{k: v.shape for k, v in std_err.items()}

shape: perc x cond

In [ ]:
statistic.mean(0)

In [ ]:
pvalue

In [ ]:
multipletests(pvalue.ravel(), method='bonferroni')

## Save

In [ ]:
final = {
    'pvalue': pvalue,
    'statistic': statistic,
    'scores_dict': scores,
    'std_err_dict': std_err,
}
np.save(pjoin(fig_dir, f'4x_n{nn}.npy'), final)